# NADAC Insights

In [1]:
# Importing libraries
import pandas as pd
import numpy as np

## Importing the Data

In [32]:
# Using the API to access the JSON and then converting it to a pandas dataframe
df = pd.read_json('https://data.medicaid.gov/resource/ckd4-m8y6.json')
df.head()

,ndc_description,ndc,nadac_per_unit,effective_date,pricing_unit,pharmacy_type_indicator,otc,explanation_code,classification_for_rate_setting,as_of_date,corresponding_generic_drug_nadac_per_unit,corresponding_generic_drug_effective_date
0,12-HR DECONGEST 120 MG CAPLET,113005452,0.30601,2021-03-17T00:00:00.000,EA,C/I,Y,1,G,2021-04-14T00:00:00.000,NaN,NaN
1,12-HR DECONGEST 120 MG CAPLET,36800005452,0.30601,2021-03-17T00:00:00.000,EA,C/I,Y,1,G,2021-04-14T00:00:00.000,NaN,NaN
2,12-HR DECONGEST 120 MG CAPLET,36800005460,0.30601,2021-03-17T00:00:00.000,EA,C/I,Y,1,G,2021-04-14T00:00:00.000,NaN,NaN
3,12HR NASAL DECONGEST ER 120 MG,24385005452,0.30601,2021-03-17T00:00:00.000,EA,C/I,Y,1,G,2021-04-14T00:00:00.000,NaN,NaN
4,12HR NASAL DECONGEST ER 120 MG,70000047501,0.30601,2021-03-17T00:00:00.000,EA,C/I,Y,1,G,2021-04-14T00:00:00.000,NaN,NaN


In [33]:
df.count()

ndc_description                              1000
ndc                                          1000
nadac_per_unit                               1000
effective_date                               1000
pricing_unit                                 1000
pharmacy_type_indicator                      1000
otc                                          1000
explanation_code                             1000
classification_for_rate_setting              1000
as_of_date                                   1000
corresponding_generic_drug_nadac_per_unit      29
corresponding_generic_drug_effective_date      29
dtype: int64

#### I realize that at least without signing up and getting an app token, I only have access to the first 1000 entries of the data set. 

In [216]:
# Loading in the dataset after downloading it as a CSV
df = pd.read_csv('NADAC_as_of_2021-04-14.csv')

In [217]:
df.count()

NDC Description                              24947
NDC                                          24947
NADAC_Per_Unit                               24947
Effective_Date                               24947
Pricing_Unit                                 24947
Pharmacy_Type_Indicator                      24947
OTC                                          24947
Explanation_Code                             24947
Classification_for_Rate_Setting              24947
Corresponding_Generic_Drug_NADAC_Per_Unit      713
Corresponding_Generic_Drug_Effective_Date      713
As of Date                                   24947
dtype: int64

## Prepping the Data

In [218]:
# Converting the two series containing dates to date time objects for convenience
df['Effective_Date'] = pd.to_datetime(df['Effective_Date'])
df['Corresponding_Generic_Drug_Effective_Date'] = pd.to_datetime(df['Corresponding_Generic_Drug_Effective_Date'])

In [219]:
# All entries have the same indicator (Data was collected from surveys of Chain/Independent pharmacies) and "As of date"
print(df['Pharmacy_Type_Indicator'].unique())
print(df['As of Date'].unique())

['C/I']
['04/14/2021']


In [220]:
df = df.drop(['Pharmacy_Type_Indicator', 'As of Date'], axis=1)

In [221]:
# For the sake of simplicity, I'm going to exclude entries priced by ML or GM and only study entries priced by each. 
# Given more time, I would explore the entries with ML and GM and possible effects the pricing unit has on the quality 
# of the deal.
df = df[df['Pricing_Unit'] == 'EA'].drop(['Pricing_Unit'], axis=1)

In [222]:
df = df.sort_values(by = ['NADAC_Per_Unit'])
# Creating a seperate df for entries with a corresponding generic drug
drugsWithCorrGeneric = df.dropna()

In [223]:
df

,NDC Description,NDC,NADAC_Per_Unit,Effective_Date,OTC,Explanation_Code,Classification_for_Rate_Setting,Corresponding_Generic_Drug_NADAC_Per_Unit,Corresponding_Generic_Drug_Effective_Date
19544,ASPIRIN 325 MG TABLET,904680940,0.00963,2021-03-17,Y,1,G,NaN,NaT
14448,ASPIRIN 325 MG TABLET,904674460,0.00963,2021-03-17,Y,1,G,NaN,NaT
9169,ASPIRIN 325 MG TABLET,536330510,0.00963,2021-03-17,Y,1,G,NaN,NaT
5883,ASPIRIN 325 MG TABLET,70000050701,0.00963,2021-03-17,Y,1,G,NaN,NaT
14410,ASPIRIN 325 MG TABLET,51645071601,0.00963,2021-03-17,Y,1,G,NaN,NaT
...,...,...,...,...,...,...,...,...,...
18488,LUPRON DEPOT 11.25 MG 3MO KIT,74366303,4284.62096,2021-01-01,N,"4, 5",B,NaN,NaT
625,CIMZIA 2X200 MG/ML SYRINGE KIT,50474071079,4693.24960,2021-01-01,N,"4, 5",B,NaN,NaT
20333,LUPRON DEPOT 22.5 MG 3MO KIT,74334603,5147.68265,2021-01-01,N,"4, 5",B,NaN,NaT
12231,AVONEX PEN 30 MCG/0.5 ML KIT,59627033304,6995.38950,2021-01-01,N,"4, 5",B,NaN,NaT


## Analyzing the Data

In [125]:
print(df.groupby('OTC')['NADAC_Per_Unit'].mean())
print(df.groupby('OTC')['NADAC_Per_Unit'].count())

OTC
N    6.542563
Y    0.349327
Name: NADAC_Per_Unit, dtype: float64
OTC
N    19028
Y     1799
Name: NADAC_Per_Unit, dtype: int64


#### Hypothesis: Over the counter drugs are cheaper per unit.

In [225]:
print(df.groupby('Classification_for_Rate_Setting')['NADAC_Per_Unit'].mean())
print(df.groupby('Classification_for_Rate_Setting')['NADAC_Per_Unit'].count())
print(drugsWithCorrGeneric['NADAC_Per_Unit'].mean())
print(drugsWithCorrGeneric['Corresponding_Generic_Drug_NADAC_Per_Unit'].mean())

Classification_for_Rate_Setting
B         75.266626
B-ANDA    12.962424
G          1.138836
Name: NADAC_Per_Unit, dtype: float64
Classification_for_Rate_Setting
B          1359
B-ANDA       56
G         19412
Name: NADAC_Per_Unit, dtype: int64
19.374043651050084
4.904571373182552


#### Hypothesis: Brand name and B_ANDA drugs are more expensive per unit than generic drugs. The average brand name drug is 75 dollars each while the average generic drug is a little over 1 dollar. Also, the average difference between price of brand name drugs with a corresponding generic drug and its corresponding generic drug is about 4X.

In [230]:
print(df[df['Explanation_Code'].str.contains('1')]['NADAC_Per_Unit'].mean())
print(df[df['Explanation_Code'].str.contains('1')]['NADAC_Per_Unit'].count())
print(df[df['Explanation_Code'].str.contains('2')]['NADAC_Per_Unit'].mean())
print(df[df['Explanation_Code'].str.contains('2')]['NADAC_Per_Unit'].count())
print(df[df['Explanation_Code'].str.contains('3')]['NADAC_Per_Unit'].mean())
print(df[df['Explanation_Code'].str.contains('3')]['NADAC_Per_Unit'].count())
print(df[df['Explanation_Code'].str.contains('4')]['NADAC_Per_Unit'].mean())
print(df[df['Explanation_Code'].str.contains('4')]['NADAC_Per_Unit'].count())
print(df[df['Explanation_Code'].str.contains('5')]['NADAC_Per_Unit'].mean())
print(df[df['Explanation_Code'].str.contains('5')]['NADAC_Per_Unit'].count())
print(df[df['Explanation_Code'].str.contains('6')]['NADAC_Per_Unit'].mean())
print(df[df['Explanation_Code'].str.contains('6')]['NADAC_Per_Unit'].count())

0.9187595774957431
18201
53.42925405797102
828
55.34207885714286
35
35.29301300056723
1763
285.01898952830186
212
5.72390891957758
1231


#### Hypothesis: There is a strong relationship between the average cost of a drug and the explanation codes. 

In [224]:
df['NADAC_Per_Unit'].corr(pd.to_numeric(df['Effective_Date']))

-0.078830668873629

#### There isn't evidence for change in average price per unit relative to the effective date over the last year

## Answers to questions:

Why did you think stat x was important?
    * NADAC_Per_Unit was the most important in my opinion, as it relates to the price of a drug.
    * OTC, Explanation_Code, and Classification_for_Rate_Setting all seemed to correlate to the NADAC_Per_Unit.
    * I wish I could have done more with NDC, as it would offer some more information about the drug.
    * NDC_Description is important because it allows you to save money by comparing interchangable drugs. If I was to continue with this project, this would be my next step.

What do you wish the data had?  
    * I wish the NDC was split up by dashes so I can tell the labeler from the product and package code. I might be able to find some insights by comparing segments of the NDC.
    * I wish the data had information on how much a dosage is for each drug, and how many dosages are need daily so that I can create a better measurement for how expensive each drug actually is.

What piece of information is the most important?  Why?
    * The most important piece of information is the price of the drug. Without the price of the drug, it's hard to gain insights that may save money.

How did you import the data?
    * First, I tried accessing the API and converting the JSON to a pandas Dataframe. I found that without signing up and gaining an access key, I was only allowed limited API calls, and a portion of the Dataframe. Because of this, I decided to download the data in the form of a CSV then import it as a pandas Dataframe.

Why go that route?  
    * Without gaining a key, I couldn't get the entire data set through an API call. By reading the CSV into a pandas Dataframe, I'm able to easily manipulate, analyze, and even model the data if I need to in order to gain insights.

Any downfalls you can think of?  
    * Data is read into a Dataframe off my laptop so the project wouldn't be easily collaborated on.
    * If the data set was large, I wouldn't want an extra copy of it in CSV form on my laptop.
    * I have to have the entire set on my local machine, rather than querying part of the set at a time.
    
    
What would you improve upon version 2?
    * I would sign up for an access key so I could get the data directly through an API call.
    * I would perform more statistical analysis to actually back up my hypotheses. 
    * I would check different versions of the same kind of drug. For example, I would check different forms and sizes of Aspirin and check which one is the cheapest. I'd look to see if I can find patterns beyond Aspirin. For example, maybe larger tablets of the same drug usually cost less per gram.
    * I'd look deeper into NDC to see if I can find insights on different labelers, drug strengths, and packaging types.
    * I'd learn more about pharmacutical prices so I can clean the data better. Some entries might be scewing results.
    * I would find a better way to regulate the NADAC_Per_Unit column. Price per each isn't very useful since we don't know how much a unit of "each" is. GM and ML didn't make much sense to me ether though as different medicines require different dosages.

How would you automate downloading / updating the data?
    * To update the data, I would obtain a key so that I could access the entire data set and make unlimited api calls, then I would run the python script to read in the updated data to a dataframe from the API. I could go a step further and have the script run automatically at a given time. (By updating I'm assuming you mean updating the data on my end and not writing to a database)

What other questions would you ask from this data set if you had more time to explore?
    * I'd question how the NADAC_Per_Unit column relates to how good of a deal each entry is.
    * I'd try to find correlation between qualities in the NDC_Description column and the price
    * I'd try to find differences in prices between entries with similar NDC_Descriptions
